In [2]:
import tkinter as tk
import colormaps as cmaps
import matplotlib.pyplot as plt
import plotly.express as px
import random as rand
import tempfile
import colormaps as cmaps
import math
import numpy as np
import pandas as pd
import scipy.optimize as opt
import openpyxl
import time
import threading
import os
import sympy

from tkinter import ttk
from tkinter import Label, Entry, font
from tkinter import filedialog as fd
from tkinter import filedialog
from tkinter import *
from tkinter import font
from colormaps.utils import show_cmaps_collection
from tkinter import messagebox
from matplotlib.figure import Figure
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg as FigureCanvas
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from PIL import ImageGrab
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D
from reportlab.pdfgen import canvas as pdf_canvas
from sympy import symbols, Eq, solve
from io import BytesIO
from scipy.optimize import fsolve
from PIL import Image, ImageTk

excel_file = "El_constants.xlsx"
workbook = openpyxl.load_workbook(excel_file, data_only=True)
sheet = workbook.active

def MainFigure(distance_scale=10, change=0):
    # Create a Matplotlib figure
    figure = Figure(figsize=(6, 6))
    canvas = FigureCanvas(figure, main)
    
    substrate_x = [9, 25, 25, 9, 9]
    substrate_y = [13.5, 13.5, 20.5, 20.5, 13.5]
    substrate_z = [12.5 + distance_scale, 12.5 + distance_scale, 12.5 + distance_scale, 12.5 + distance_scale, 12.5 + distance_scale]

    sub_x = [16.7, 17.3, 17.3, 16.7, 16.7]
    sub_y = [20.5, 20.5, 20.5, 20.5, 20.5]
    sub_z = [0,0, 12.5+distance_scale, 12.5+distance_scale, 0]
    
    sourceh_x = [15, 19, 19, 15, 15]
    sourceh_y = [16, 16, 17.5, 17.5, 16]
    sourceh_z = [12.5, 12.5, 12.5, 12.2, 12.5]

    radius = 0.75
    height_z = 12.52
    cil_z = np.linspace(0, height_z, 50)
    theta = np.linspace(0, 2*np.pi, 50)

    center_x = 15
    center_y = 16.75
    theta_grid, z_grid=np.meshgrid(theta, cil_z)
    x_grid = radius*np.cos(theta_grid) + center_x
    y_grid = radius*np.sin(theta_grid) + center_y

    center_x1 = 19
    center_y1 = 16.75
    x_grid1 = radius*np.cos(theta_grid) + center_x1
    y_grid1 = radius*np.sin(theta_grid) + center_y1

    ax = figure.add_subplot(111, projection='3d')
    ax.plot(substrate_x, substrate_y, substrate_z, color='cadetblue', linewidth=2, label='Substrate')
    ax.plot(sourceh_x, sourceh_y, sourceh_z, color='darkgrey', linewidth=2, label='Source holder')
    ax.plot(sub_x, sub_y, sub_z, color='darkgrey', linewidth=2)
    ax.plot_surface(x_grid, y_grid, z_grid, alpha=0.5, color = 'dimgrey')
    ax.plot_surface(x_grid1, y_grid1, z_grid, alpha=0.5, color = 'dimgrey')

    if change != 0:  
        ax.scatter(17,17,12.6, s=1+change*6, color='black', label='Source') 

    substrate_verts = [list(zip(substrate_x, substrate_y, substrate_z))]
    sub_verts = [list(zip(sub_x, sub_y, sub_z))]
    sourceh_verts = [list(zip(sourceh_x, sourceh_y, sourceh_z))]
    substrate_surface = Poly3DCollection(substrate_verts, color='turquoise', alpha=0.5)
    sourceh_surface = Poly3DCollection(sourceh_verts, color='darkgrey', alpha=0.5)
    sub_surface = Poly3DCollection(sub_verts, color ='darkgrey', alpha=0.5)

    ax.add_collection3d(substrate_surface)
    ax.add_collection3d(sourceh_surface)
    ax.add_collection3d(sub_surface)

    ax.set_xlabel('X (cm)')
    ax.set_ylabel('Y (cm)')
    ax.set_zlabel('Z (cm)')
    ax.legend()
    ax.set_title('Thermal Evaporation Method/Particle Deposition')
    ax.set_xlim(0, 34)
    ax.set_ylim(0, 34)
    ax.set_zlim(0, 34)
    ax.grid(False)

    ax.set_xticks([0,5, 10, 15,20, 25, 30])  
    ax.set_yticks([0,5, 10, 15,20, 25, 30])  
    ax.set_zticks([0,5, 10, 15,20, 25, 30])

    canvas.get_tk_widget().place(relx=0.9, rely=0.1, anchor=tk.NE)
    canvas.draw()
    
def update_distance(event):
    if event.keysym == 'Return':
        try:
            distance_value = float(distance_var.get())
            if 0 < distance_value <= 27:
                MainFigure(distance_value, change=0)
        except ValueError:
            pass

def update_mass(event):
    if event.keysym == 'Return':
        try:
            mass_value = float(mass_var.get())
            element_input = element_var.get()
            if element_input:
                element_found = False
                for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=2, max_col=2, values_only=True):
                    if element_input in row:
                        element_found = True
                        try:
                            if mass_value != 0:  
                                MainFigure(float(distance_var.get()), change=mass_value)
                            else:
                                MainFigure(float(distance_var.get()), change=0)
                        except ValueError:
                            pass
                        break
                if not element_found:
                    messagebox.showwarning("Warning", "Element not found in the database.")
        except ValueError:
            pass


def show_message():
    messagebox.showinfo("Info", "The elements that you can choose as a source are the ones that we have in our database. See Database to further details.")
    
def show_message1():
    messagebox.showinfo("Info", "Particle distribution options: Isotropic, Truncated isotropic, Anisotropic.")

global lenght
lenght = 16 #cm

global width
width = 7 #cm

def Interface():
    #Tkinter window
    global main
    main = tk.Tk()
    main.geometry("3000x1080")
    main.title('Thermal Evaporation Method - Thickness Estimation')
    main.iconbitmap("icone.ico")

    my_font = font.Font(size=13)

    
    def go_to_next_element_e(event):
        if event.widget == element:
            mass.focus_set()  
    
    
    def go_to_next_element(event):
        event.widget.tk_focusNext().focus()
        
    def update_distance_and_next(event):
        if event.keysym == 'Return':
            update_distance(event)  
            go_to_next_element(event)  

    def update_mass_and_next(event):
        if event.keysym == 'Return':
            update_mass(event)  
            go_to_next_element(event)
            
    def go_to_next_element_m(event):
        if event.widget == mass:
            area.focus_set()  
    
    def dis_next(event):
        if d_var.get()=='Isotropic' or d_var.get()=='Anisotropic' or d_var.get()=='Truncated isotropic':
            event.widget.tk_focusNext().focus()
        else:
            messagebox.showwarning("Warning", "Not valid distribution.")
            
    def missing_p():
        if not d_var.get() or not distance_var.get() or not area_var.get() or not element_var.get() or not mass_var.get or not press_var.get():
            messagebox.showwarning("Warning", "Missing parameters.")
        else:
            results()
        

    frame_input = tk.Frame(main)
    frame_input.grid(row=0, column=0, padx=0, pady=0)
    label_input = tk.Label(frame_input, text="Inputs required for the thermal evaporation method", font=("Arial", 14, 'bold'))
    label_input.grid(row=0, column=0, sticky='W')

    frame_dist = tk.Frame(main)
    frame_dist.grid(row=4, column=0, padx=10, pady=10)
    label_dist = tk.Label(frame_dist, text="Distance between the source and the surface (in cm):")
    label_dist.grid(row=0, column=0, sticky='W')

    global distance_var
    distance_var = tk.StringVar()
    distance_var.set('10')

    global distance
    distance = tk.Entry(frame_dist, textvariable=distance_var)
    distance.grid(row=0, column=1, sticky='W')
    distance.bind("<KeyRelease>", update_distance_and_next)  

    frame_element = tk.Frame(main)
    frame_element.grid(row=8, column=0, padx=0, pady=0)
    label_element = tk.Label(frame_element, text="Source (insert element (ex: Al))")
    label_element.grid(row=0, column=0, sticky='E')

    global element_var
    element_var = tk.StringVar()
    element_var.set('')

    global element
    element = tk.Entry(frame_element, textvariable=element_var)
    element.grid(row=0, column=1, padx=0, pady=0,ipadx=0, ipady=0)
    element.bind("<KeyRelease>", update_mass_and_next)

    frame_mass = tk.Frame(main)
    frame_mass.grid(row=12, column=0, padx=10, pady=10)
    label_mass = tk.Label(frame_mass, text="Mass of the source (in g):")
    label_mass.grid(row=0, column=0, sticky='w')

    global mass_var
    mass_var = tk.StringVar()
    mass_var.set('0')

    global mass
    mass = tk.Entry(frame_mass, textvariable=mass_var)
    mass.grid(row=0, column=1, sticky='w')
    mass.bind("<KeyRelease>", update_mass_and_next)

    frame_area = tk.Frame(main)
    frame_area.grid(row=16, column=0, padx=10, pady=0)
    label_area = tk.Label(frame_area, text="Area of the source (in cm²):")
    label_area.grid(row=0, column=0, sticky='w')

    global area_var
    area_var = tk.StringVar()
    area_var.set('0')

    global area
    area = tk.Entry(frame_area, textvariable=area_var)
    area.grid(row=0, column=1, sticky='w')
    area.bind('<Return>', go_to_next_element)

    frame_press = tk.Frame(main)
    frame_press.grid(row=20, column=0, padx=0, pady=10)
    label_press = tk.Label(frame_press, text="Pressure inside the vacuum chamber (in mbar):")
    label_press.grid(row=0, column=0, sticky='E')

    global press_var
    press_var = tk.StringVar()
    press_var.set('0')

    global press
    press = tk.Entry(frame_press, textvariable=press_var)
    press.grid(row=0, column=1, padx=0, pady=0,ipadx=0, ipady=0)
    press.bind('<Return>', go_to_next_element)
    
    frame_temp = tk.Frame(main)
    frame_temp.grid(row=24, column=0, padx=0, pady=0)
    label_temp = tk.Label(frame_temp, text="Minimum temperature needed to evaporate the source (in K):")
    label_temp.grid(row=0, column=0, sticky='E')

    frame_d = tk.Frame(main)
    frame_d.grid(row=25, column=0, padx=0, pady=10)
    label_d = tk.Label(frame_d, text="Particle distribution")
    label_d.grid(row=0, column=0, sticky='E')

    global d_var
    d_var = tk.StringVar()
    d_var.set('')

    global d
    d = tk.Entry(frame_d, textvariable=d_var)
    d.grid(row=0, column=1, padx=0, pady=0,ipadx=0, ipady=0)
    
    d.bind('<Return>', dis_next)
    
    exclamation_button = tk.Button(main, text="!", width=1, height=1, font=("Helvetica", 11), command=show_message)
    exclamation_button.grid(row=8, column=0, padx=(0, 0), sticky="E")  
    
    exclamation_button2 = tk.Button(main, text="!", width=1, height=1, font=("Helvetica", 11), command=show_message1)
    exclamation_button2.grid(row=25, column=0, padx=(0, 0), sticky="E") 

    frame_filmes = Frame(main)
    frame_filmes.place(relx=0.19, rely=0.6, anchor="center")

    img_1 = Image.open("lamelas.png")
    resize_image1 = img_1.resize((300, 120))

    image_1 = ImageTk.PhotoImage(resize_image1)
    label_f = Label(frame_filmes, image = image_1)
    label_f.pack()
    

    MainFigure()  

    #######################################################################################################################
    ########################################  MINIMUM TEMPERATURE DETERMINATION  ##########################################
    #######################################################################################################################
    def temperature():
        A, B, C, D, P= symbols('A B C D P')
        if press_var.get():
            P=press_var.get()
            for row in range(1, sheet.max_row + 1):
                if element_var.get() == sheet.cell(row=row, column=2).value:
                    A = sheet.cell(row=row, column=6).value
                    B = sheet.cell(row=row, column=7).value
                    C = sheet.cell(row=row, column=8).value
                    D = sheet.cell(row=row, column=9).value
                    break    

        return A, B, C, D, P
    
   
    def calculate_temperature():
        params = temperature()

        if params:
            A, B, C, D, P = params
            initial_guess = 100
            T = fsolve(lambda T: float(A) + float(B)/T + float(C) * math.log10(T) + float(D)/(T**3) - math.log10(float(P)*0.000986923267), initial_guess)

            return T[0]
        else:
            return None
        
   
    label_tempe = tk.Label(main, text="", width=8) 
    label_tempe.grid(row=24, column=0, padx=0, pady=0, sticky='E')

    def update_temperature_label():
        temperature_value = calculate_temperature()
        
        if temperature_value is not None:
            formatted_text = f"{temperature_value:.1f} K"
            label_tempe.config(text=formatted_text)
            label_tempe.grid()  # Show the label
        else:
            label_tempe.grid_remove()  # Hide the label
            
    ################################################################    
    update_temp_button = tk.Button(main, text="Calculate Temperature", command=update_temperature_label)
    update_temp_button.grid(row=20, column=1, padx=15, pady=0)
    
    #######################################################################################################################
    ############################################  Thickness determination   ############################################
    #######################################################################################################################
    def Thick(Me, h):
        N_point = 12
        largura = [6.5*p/(2*N_point) -3.25 for p in range(0, 2*N_point+1)]
        comprimento = [15.7*p/(6*N_point) - 7.85 for p in range(0, 6*N_point+1)]
        thickness_matrix = np.zeros((len(largura), len(comprimento)), dtype=np.float64)
        incerteza_matrix = np.zeros((len(largura), len(comprimento)), dtype=np.float64)
        delta_Me = 0.0005  
        delta_h = 0.2 
        delta_a = 0.1
        delta_comp = 0.1*np.sqrt((1/len(comprimento))*(1/len(comprimento)) + 1)   
        delta_larg = 0.1*np.sqrt((1/len(largura))*(1/len(largura)) + 1)   
        lista = []
        if float(mass_var.get())!=0 and float(distance_var.get())!=0 and d_var.get():
            aperture = 1
            exp1 = 1
            exp2 = 1
            if d_var.get() =='Isotropic':
                aperture = 2
                exp1 = 1
                exp2 = 1.5
            if d_var.get() =='Truncated isotropic':
                aperture = 1.44
                exp1 = 1
                exp2 = 1.5
            elif d_var.get() =='Anisotropic':
                aperture = 1
                exp1 = 2
                exp2 = 2
            if aperture==1.44:
                delta_aperture = np.pi*2.5/180
            else:
                delta_aperture = 0
            for i, larg in enumerate(largura):
                for j, comp in enumerate(comprimento):
                    h = float(distance_var.get())
                    Me = float(mass_var.get())
                    t = 0.9* Me * h**exp1 / (aperture * np.pi * (h**2 + comp**2 + larg**2)**exp2) * 1000 * 1000  # ug / cm^2
                    thickness_matrix[i, j] = t 
                    delta_t_Me = (0.9*h ** exp1) / (aperture * np.pi * (h ** 2 + comp ** 2 + larg ** 2) ** exp2) * 10**6
                    delta_t_a = (Me*h ** exp1) / (aperture * np.pi * (h ** 2 + comp ** 2 + larg ** 2) ** exp2) * 10**6
                    delta_t_h = ((0.9*Me * exp1 * h ** (exp1 - 1) * aperture * np.pi * (h ** 2 + comp ** 2 + larg ** 2) ** exp2) - (0.9*2 * exp2 * h * Me * aperture * np.pi * (h ** 2 + comp ** 2 + larg ** 2)**(exp2-1) * h ** exp1)) /((aperture * np.pi * ((h ** 2 + comp ** 2 + larg ** 2) ** exp2)) ** 2) * 10**6
                    delta_t_comp = (-aperture * 0.9 * exp2 * np.pi * (h ** 2 + comp ** 2 + larg ** 2)**(exp2-1) * 2 * comp * Me * h ** exp1) / ((aperture * np.pi * ((h ** 2 + comp ** 2 + larg ** 2) ** exp2)) ** 2) * 10**6
                    delta_t_larg = (-aperture * 0.9*  exp2 * np.pi * (h ** 2 + comp ** 2 + larg ** 2)**(exp2-1) * 2 * larg * Me * h ** exp1) / ((aperture * np.pi * ((h ** 2 + comp ** 2 + larg ** 2) ** exp2)) ** 2) * 10**6
                    delta_t_ap = (-Me * 0.9 * h ** exp1 * np.pi * (h**2 + comp**2 + larg**2)**exp2) / ((aperture * np.pi * ((h ** 2 + comp ** 2 + larg ** 2) ** exp2)) ** 2) * 10**6
                    uncertainty = np.sqrt((delta_t_Me * delta_Me) ** 2 + (delta_t_h * delta_h) ** 2 + (delta_t_comp * delta_comp) ** 2 + (delta_t_larg * delta_larg) ** 2 + (delta_t_ap * delta_aperture) ** 2+ (delta_t_a * delta_a) ** 2)
                    incerteza_matrix[i,j] = uncertainty
                    lista.append([comp, larg, t, uncertainty])
            
        return lista

    
    frame_thick = tk.Frame(main)
    frame_thick.grid(row=49, column=0, padx=0, pady=0)
    label_thick = tk.Label(frame_thick, text="")
    label_thick.grid(row=0, column=0, sticky='E')
    
    global variation_plot_visible, canvas
    variation_plot_visible = False  
    canvas = None 
    
    def flux():
        if float(mass_var.get())!=0 and float(distance_var.get())!=0 and d_var.get() and float(area_var.get())!=0:
            temp = calculate_temperature()
        for row in range(1, sheet.max_row + 1):
            if element_var.get() == sheet.cell(row=row, column=2).value:
                m = sheet.cell(row=row, column=3).value
        fl = 6.022E23/(2*math.pi*m*8.31*temp)**0.5*1.661E-27*1000
        t = float(mass_var.get())/(float(area_var.get())*fl)
        return t
    
    def substrato():
        fig = plt.figure(figsize=(10,8))
        dados = Thick(float(mass_var.get()), distance_var.get())
        x = [coord[0] for coord in dados]
        y= [coord[1] for coord in dados]
        esp  = [coord[2] for coord in dados]
        scatter = plt.scatter(x, y, c=esp, s=700, cmap=cmaps.bubblegum, marker = 's')
        cbar = plt.colorbar(scatter, shrink=0.45)
        cbar.set_label('Thickness t (\u03BCg/cm\u00b2)')
        plt.axvline(x = 0, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = 16/6, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = 16/6*2, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = 16/6*3, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = -16/6, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = -16/6*2, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = -16/6*3, color = 'black', label = 'axvline - full height')
        rectangle = plt.Rectangle((-8,-3.5), 16, 7, fc='azure',ec="azure", zorder=-1)
        plt.gca().add_patch(rectangle)
        plt.axis('scaled')
        plt.ylim(-3.5,3.5)
        plt.xlim(-8,8)
        plt.xlabel('x-pos (cm)')
        plt.ylabel('y-pos (cm)')
        plt.savefig("thickness.png", bbox_inches='tight') 
        plt.close()
        return fig
    
    def inc():
        fig = plt.figure(figsize=(10,8))
        dados = Thick(float(mass_var.get()), distance_var.get())
        x = [coord[0] for coord in dados]
        y= [coord[1] for coord in dados]
        esp  = [coord[3] for coord in dados]
        scatter = plt.scatter(x, y, c=esp, s=700, cmap=cmaps.l_purplowsat3, marker = 's')
        cbar = plt.colorbar(scatter, shrink=0.45)
        cbar.set_label('Uncertainty (\u03BCg/cm\u00b2)')
        plt.axvline(x = 0, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = 16/6, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = 16/6*2, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = 16/6*3, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = -16/6, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = -16/6*2, color = 'black', label = 'axvline - full height') 
        plt.axvline(x = -16/6*3, color = 'black', label = 'axvline - full height')
        rectangle = plt.Rectangle((-8,-3.5), 16, 7, fc='azure',ec="azure", zorder=-1)
        plt.gca().add_patch(rectangle)
        plt.axis('scaled')
        plt.ylim(-3.5,3.5)
        plt.xlabel('x-pos (cm)')
        plt.ylabel('y-pos (cm)')
        plt.xlim(-8,8)
        plt.savefig("uncertainties.png", bbox_inches='tight')  
        plt.close()
        return fig
    

    def save_to_pdf():
        file_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        if file_path:
            try:
                c = pdf_canvas.Canvas(file_path, pagesize=letter)

                c.drawString(160, 720, "Thermal Evaporation Method Thickness Results")
                c.drawString(10, 680, "Distance between source and substrate: " + distance_var.get() + " cm")
                c.drawString(10, 660, "Element: " + element_var.get())
                c.drawString(10, 640, "Mass: " + mass_var.get() + " g")
                c.drawString(10, 620, "Area: " + area_var.get() + " cm\u00b2")
                c.drawString(10, 600, "Pressure: " + press_var.get() + " mbar")
                c.drawString(10, 580, "Particle distribution: " + d_var.get())
                c.drawString(10, 540,"Time needed to evaporate the source: " + str(int(flux())) + " s")
                c.drawString(10, 510,"Thickness distribution on substrate")
                c.drawString(10, 300,"Uncertainty of Thickness distribution on substrate")
                             
                img = Image.open('thickness.png')
                img1 = Image.open('uncertainties.png')
                c.drawImage('thickness.png', 120, 350, width=320, height=110) 
                c.drawImage('uncertainties.png', 120, 150, width=320, height=110) 
                messagebox.showinfo("Success", "PDF saved successfully!")
                c.save()
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {str(e)}")
        
    def results():
        substrato()
        inc()
        global window 
        window = tk.Toplevel(main)
        window.title("Results")
        window.geometry("760x770") 
        window.resizable(False,False)
        window.configure(bg="white") 
        
        frame = tk.Frame(window)
        frame.pack(fill=tk.BOTH, expand=1)

        canvas_tk = tk.Canvas(frame, bg="white")
        canvas_tk.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        
        my_scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL, command=canvas_tk.yview)
        my_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        canvas_tk.configure(yscrollcommand=my_scrollbar.set)
        canvas_tk.bind('<Configure>', lambda e: canvas_tk.configure(scrollregion=canvas_tk.bbox("all")))
        
        content_frame = tk.Frame(canvas_tk, bg="white")
        canvas_tk.create_window((0, 0), window=content_frame, anchor='nw')

        title_label = tk.Label(content_frame, text="Thermal Evaporation Method Thickness Results", font=("Arial", 12), bg="white", fg="black", anchor=CENTER)
        title_label.pack(padx=200, pady=(0, 20))
        
        distance_value = distance_var.get()
        distance_label = tk.Label(content_frame, text="Distance between source and substrate: " + distance_value + " cm",font=("Arial", 11), bg="white", fg="black")
        distance_label.pack(anchor="w")
        
        element_value = element_var.get()
        element_label = tk.Label(content_frame, text="Element: " + element_value,font=("Arial", 11), bg="white", fg="black")
        element_label.pack(anchor="w")

        mass_value = mass_var.get()
        mass_label = tk.Label(content_frame, text="Mass: " + mass_value + " g", font=("Arial", 11), bg="white", fg="black")
        mass_label.pack(anchor="w")
        
        area_value = area_var.get()
        area_label = tk.Label(content_frame, text="Area: " + area_value + " cm\u00b2", font=("Arial", 11), bg="white", fg="black")
        area_label.pack(anchor="w")
        
        pressure_value = press_var.get()
        pressure_label = tk.Label(content_frame, text="Pressure: " + pressure_value + " mbar", font=("Arial", 11), bg="white", fg="black")
        pressure_label.pack(anchor="w")
        
        d_value = d_var.get()
        d_label = tk.Label(content_frame, text="Particle distribution: " + d_value, font=("Arial", 11), bg="white", fg="black")
        d_label.pack(anchor="w")
        
        t_value = str(int(flux()))
        t_label = tk.Label(content_frame, text="Time needed to evaporate the source: " + t_value + " s", font=("Arial", 11), bg="white", fg="black")
        t_label.pack(anchor="w")
    
        label_plot1 = tk.Label(content_frame, text="Thickness distribution on substrate", font=("Arial", 11), bg="white", fg="black")
        label_plot1.pack(anchor="w", pady=10)
        
        img1 = Image.open('thickness.png')
        img_tk = ImageTk.PhotoImage(img1)
        
        label = tk.Label(content_frame, image=img_tk, bg="white", anchor= CENTER)
        label.pack()
        label.image = img_tk
        
        label_plot2 = tk.Label(content_frame, text="Uncertainty of Thickness distribution on substrate", font=("Arial", 11), bg="white", fg="black")
        label_plot2.pack(anchor="w")
        
        img2 = Image.open('uncertainties.png')
        img2_tk = ImageTk.PhotoImage(img2)
        
        label2 = tk.Label(content_frame, image=img2_tk, bg="white")
        label2.pack(fill="both", expand=True, padx=20, pady=20)
        label2.image = img2_tk
        
        save_button = tk.Button(window, text="Save as PDF", command=save_to_pdf)
        save_button.pack()
        close_button = tk.Button(window, text="Close", command=window.destroy)
        close_button.pack(pady=10)

    button = tk.Button(main, text="Simulate deposition", fg = "gray", font=("Arial", 16, 'bold'), command=lambda: [f() for f in [missing_p]])
    button.grid(row=26, column=0, padx=10, pady=5)

    def reset_values():
        element_var.set('')
        mass_var.set('0')
        distance_var.set('10')
        area_var.set('0')
        press_var.set('0')
        d_var.set('')
        label_tempe.config(text='') 
        if os.path.exists("thickness.png"):
            os.remove("thickness.png")
        if os.path.exists("uncertainties.png"):
            os.remove("uncertainties.png")
        
        mainMenu() 
        Interface

    reset_button = tk.Button(main, text="Reset", command=reset_values)
    reset_button.grid(row=28, column=0, padx=10, pady=5)
    #######################################################################################################################

    for widget in [label_dist, label_element, label_mass, element, mass, distance, label_press, press, label_temp, label_thick, label_area, area, label_temp, label_tempe, label_d,d]:
        widget.config(font=my_font)

    ################################################################
    def about_popup(tks):

        popup_window = tk.Toplevel(tks)
        popup_window.title("Database")
        popup_window.geometry("1900x900")

        label = tk.Label(popup_window, text="Here you can see different parameters for a list of elements.")
        label.pack(padx=5, pady=5)

        text_widget = tk.Text(popup_window, wrap=tk.NONE,width=175, height=200)
        text_widget.pack()
        scrollbar = Scrollbar(popup_window, command=text_widget.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        text_widget.config(yscrollcommand=scrollbar.set)
        
        col_widths = [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]  
        for row in sheet.iter_rows(values_only=True):
            row_text = " | ".join(f"{col:{width}}" if col is not None else "" for col, width in zip(row, col_widths)) + "\n"
            text_widget.insert(tk.END, row_text)

        text_widget.config(state=tk.DISABLED)
        close_button = tk.Button(popup_window, text="Close", command=popup_window.destroy)
        close_button.pack(pady=10)

        return
    
    def open_pdf():
        pdf_path = 'User Manual.pdf'
        import webbrowser
        webbrowser.open(pdf_path)
    
    def user_popup():

        user_window = tk.Toplevel(main)
        user_window.title("About")
        user_window.geometry("400x150")

        label = tk.Label(user_window, text="To learn more about the interface click on the option below.")
        label.pack(padx=5, pady=5)

        button = tk.Button(user_window, text="Open User Manual as .pdf file", command=open_pdf)
        button.pack(pady=10)
        close_button = tk.Button(user_window, text="Close", command=user_window.destroy)
        close_button.pack(pady=10)

        return

    ################################################################
    def open_user_manual(event):
        if event.widget == manual_menu:  
            user_popup()
    
    def mainMenu():
        menu = tk.Menu(main)
        main.config(menu=menu)
        
        file_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label='Home', menu=file_menu, background='blue')
        file_menu.add_command(label='Reset', command=lambda: (main.destroy(), Interface()))
        file_menu.add_command(label='Exit', command=lambda: main.destroy())
        
        data_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Database", menu=data_menu)
        data_menu.add_command(label="View Database", command=lambda: about_popup(main))
        
        global manual_menu
        manual_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Manual", menu=manual_menu)
        manual_menu.add_command(label="Open User Manual", command=lambda: user_popup())
    
    workbook.close()
    mainMenu()
    main.mainloop()

Interface()